# DB matching

In [1]:
base = "data"
dataset = "nist"
kind = "in_database"

data_train_path =f"{base}/{dataset}/{kind}/train.msp"
data_val_path =f"{base}/{dataset}/{kind}/val.msp"
data_test_path =f"{base}/{dataset}/{kind}/test.msp"

max_mz = 1001

%load_ext autoreload
%autoreload 2

## Data loading

In [2]:
from matchms.importing import load_from_msp
from data_utils import spectrum_processing, get_n_samples

from scipy.sparse import csr_matrix

from sklearn.metrics.pairwise import cosine_similarity
from numpy.random import default_rng
from helpers import get_his_size, get_mz_vector

import numpy as np

In [3]:
def get_spectra(path):
    # Load data from MSP file and apply filters
    spectrums = [spectrum_processing(s, min_rel_int=None) for s in load_from_msp(path)]
    # Omit spectrums that didn't qualify for analysis
    spectrums = [s for s in spectrums if s is not None]
    # Create spectrum documents
    return spectrums
spectrums_train = get_spectra(data_train_path)
spectrums_val = get_spectra(data_val_path)
spectrums_test = get_spectra(data_test_path)

In [4]:
datasets = {
    "spec_train": spectrums_train, 
    "spec_val": spectrums_val, 
    "spec_val_5000": get_n_samples(spectrums_val, 5000),
    "spec_val_10000": get_n_samples(spectrums_val, 10000),
    "spec_test": spectrums_test, 
}


## Database matrix

In [5]:
mz_matrix_train = np.zeros(shape=(len(spectrums_train), max_mz))
for i, spec in enumerate(spectrums_train):
    mz_matrix_train[i] = get_mz_vector(spec, max_mz)

In [6]:
# get matrix for weighted cosine score
def get_weighted(matrix):
    n_sam =matrix.shape[0] 
    n_mz = matrix.shape[1]
    factors = np.tile(np.arange(n_mz), n_sam).reshape(n_sam, -1) + 1
    return (matrix**0.5) * factors

weighted_cosine_mz_matrix = get_weighted(mz_matrix_train)

In [7]:
sparse_weighted_mz_matrix = csr_matrix(weighted_cosine_mz_matrix)
sparse_mz_matrix = csr_matrix(mz_matrix_train)

## Matching

In [8]:
from sklearn.neighbors import NearestNeighbors
class BaseMatcherKNN():
    def __init__(self, sparse_weighted_mz_matrix, max_mz=1001, is_weighted=False):
        self.sparse_weighted_mz_matrix = sparse_weighted_mz_matrix
        self.rng = default_rng(42)
        self.max_mz = max_mz
        self.is_weighted = is_weighted
        
        self.neigh = NearestNeighbors(n_neighbors=50, n_jobs=32, metric="cosine")
        self.neigh.fit(self.sparse_weighted_mz_matrix)
    
    def match_topk_all(self, spectrums_ref, spectrums, up_to_k, n, limit=100000):

        n_pred_ikeys_per_k = [[None for _ in range(len(spectrums[:limit]))] for _ in range(up_to_k)]
        y_ikeys = []
        for i, spec in enumerate(spectrums[:limit]):
            vect = get_mz_vector(spec, self.max_mz)
            
            m = np.zeros(shape=(up_to_k,vect.shape[0]))
            
            descending = np.argsort(spec.peaks.intensities)[::-1]
            y_ikeys.append(spec.metadata["inchikey"].split("-")[0])

            skipped = np.zeros(up_to_k) == 1
            
            for j in range(1, up_to_k+1):
                
                if len(spec.peaks.mz) <= j:
                    skipped[j-1] = True
                    continue
                    
                val_at_jth = spec.peaks.intensities[descending][j-1]
                
                    
                ## cripple vector
                m[j-1] = vect.copy()                    
                m[j-1][m[j-1] < val_at_jth] = 0
            
            if self.is_weighted:
                weighted = csr_matrix(get_weighted(m))
            else:
                weighted = csr_matrix(m)
                
            sim = cosine_similarity(weighted, self.sparse_weighted_mz_matrix)
            
            for j in range(1, up_to_k+1):
                if skipped[j-1]:
                    continue

                top_n_sindices = np.argsort(sim[j-1])[::-1][:n]

                top_n_ikeys = [spectrums_ref[i].metadata["inchikey"].split("-")[0] for i in top_n_sindices]
                n_pred_ikeys_per_k[j-1][i] = top_n_ikeys
            
        return n_pred_ikeys_per_k, y_ikeys
    
    def match_random_all(self, spectrums_ref, spectrums, probs, n, cum_level=.95, limit=100000):
        self.rng = default_rng(42)
        
        n_pred_ikeys_per_p = [[None for _ in range(len(spectrums[:limit]))] for _ in range(len(probs))]
        y_ikeys = []
        for i, spec in enumerate(spectrums[:limit]):
            
            vect = get_mz_vector(spec, self.max_mz)            
            mat = np.zeros(shape=(len(probs),vect.shape[0]))
            
            y_ikeys.append(spec.metadata["inchikey"].split("-")[0])
                      
            for m, p in enumerate(probs):        
                
                ## cripple vector
                mat[m] = vect.copy()
                                
                mask_missing = self.rng.uniform(0,1,self.max_mz) < p
                
                mat[m][mask_missing] = 0
            
            if self.is_weighted:
                weighted = csr_matrix(get_weighted(mat))
            else:
                weighted = csr_matrix(mat)
            
            top_n = self.neigh.kneighbors(weighted, n_neighbors=n, return_distance=False)
            for m, p in enumerate(probs): 
                
                top_n_sindices = top_n[m]
                top_n_ikeys = [spectrums_ref[i].metadata["inchikey"].split("-")[0] for i in top_n_sindices]
                n_pred_ikeys_per_p[m][i] = top_n_ikeys
            
        return n_pred_ikeys_per_p, y_ikeys

In [49]:
# parameters
up_to_k = 30

probs = [0, .01, .03, .05, .08, .1, .15, .2, .3, .5]

to_match = 50


In [50]:
matcher = BaseMatcherKNN(sparse_mz_matrix, max_mz=max_mz, is_weighted=False)

tested = "spec_val"
p_name = "cosine"

In [ ]:
%%time
n_pred_ikeys_per_k, y_ikeys = matcher.match_topk_all(datasets["spec_train"], datasets[tested], \
                                                     up_to_k, to_match)
np.save(f"matches/{kind}/{dataset}/{tested}/{p_name}/n_pred_ikeys_per_k.npy", n_pred_ikeys_per_k)
np.save(f"matches/{kind}/{dataset}/{tested}/{p_name}/y_ikeys.npy", y_ikeys)

In [51]:
%%time
n_pred_ikeys_per_p, y_ikeys = matcher.match_random_all(datasets["spec_train"], datasets[tested], \
                                                       probs, to_match)
np.save(f"matches/{kind}/{dataset}/{tested}/{p_name}/n_pred_ikeys_per_p.npy", n_pred_ikeys_per_p)

CPU times: user 9h 57min 6s, sys: 54min 30s, total: 10h 51min 37s
Wall time: 3h 45min 56s


In [52]:
matcher_weighted = BaseMatcherKNN(sparse_weighted_mz_matrix, max_mz=max_mz, is_weighted=True)

tested = "spec_val"
p_name = "cosine_weighted"

In [ ]:
%%time
n_pred_ikeys_per_k, y_ikeys = matcher_weighted.match_topk_all(datasets["spec_train"], \
                                                              datasets[tested], up_to_k, to_match)
np.save(f"matches/{kind}/{dataset}/{tested}/{p_name}/n_pred_ikeys_per_k.npy", n_pred_ikeys_per_k)
np.save(f"matches/{kind}/{dataset}/{tested}/{p_name}/y_ikeys.npy", y_ikeys)


In [53]:
%%time
n_pred_ikeys_per_p, y_ikeys = matcher_weighted.match_random_all(datasets["spec_train"], \
                                                                datasets[tested], probs, to_match)
np.save(f"matches/{kind}/{dataset}/{tested}/{p_name}/n_pred_ikeys_per_p.npy", n_pred_ikeys_per_p)

CPU times: user 9h 53min, sys: 1h 1min 43s, total: 10h 54min 44s
Wall time: 4h 2min 26s


## Results

In [54]:
def show_for_top(top, n_pred_ikeys_per_k, y_ikeys):
    for k in range(len(n_pred_ikeys_per_k)):
        corr = 0
        tot = 0
        for i in range(len(y_ikeys)):
            if n_pred_ikeys_per_k[k][i] is None:
                #print("short")
                continue
            if y_ikeys[i] in n_pred_ikeys_per_k[k][i][:top]:
                corr += 1
            tot += 1

        print(f"For k={k+1}, {corr/tot*100:.0f}% - {corr}/{tot} was recovered in top {top}")
        
def show_for_random(top, probs, n_pred_ikeys_per_m, y_ikeys):
    for m, p in enumerate(probs):
        corr = 0
        tot = 0
        for i in range(len(y_ikeys)):
            if n_pred_ikeys_per_m[m][i] is None:
                print("short")
                continue
            if y_ikeys[i] in n_pred_ikeys_per_m[m][i][:top]:
                corr += 1
            tot += 1

        print(f"For p={p}, {corr/tot*100:.0f}% - {corr}/{tot} was recovered in top {top}")
        

### Cosine

In [55]:
tested = "spec_val"
p_name = "cosine"

n_pred_ikeys_per_p = np.load(f"matches/{kind}/{dataset}/{tested}/{p_name}/n_pred_ikeys_per_p.npy")
n_pred_ikeys_per_k = np.load(f"matches/{kind}/{dataset}/{tested}/{p_name}/n_pred_ikeys_per_k.npy", allow_pickle=True)
y_ikeys = np.load(f"matches/{kind}/{dataset}/{tested}/{p_name}/y_ikeys.npy")

In [56]:
show_for_top(1, n_pred_ikeys_per_k, y_ikeys)

For k=1, 0% - 38/22921 was recovered in top 1
For k=2, 8% - 1846/22921 was recovered in top 1
For k=3, 24% - 5566/22921 was recovered in top 1
For k=4, 38% - 8704/22921 was recovered in top 1
For k=5, 48% - 10947/22921 was recovered in top 1
For k=6, 54% - 12266/22921 was recovered in top 1
For k=7, 58% - 13182/22921 was recovered in top 1
For k=8, 60% - 13768/22921 was recovered in top 1
For k=9, 62% - 14187/22921 was recovered in top 1
For k=10, 63% - 14488/22884 was recovered in top 1
For k=11, 64% - 14690/22854 was recovered in top 1
For k=12, 65% - 14888/22818 was recovered in top 1
For k=13, 66% - 15003/22771 was recovered in top 1
For k=14, 66% - 15085/22714 was recovered in top 1
For k=15, 67% - 15110/22640 was recovered in top 1
For k=16, 67% - 15142/22575 was recovered in top 1
For k=17, 67% - 15144/22507 was recovered in top 1
For k=18, 67% - 15128/22415 was recovered in top 1
For k=19, 68% - 15093/22305 was recovered in top 1
For k=20, 68% - 15071/22206 was recovered in top

In [57]:
show_for_random(1, probs, n_pred_ikeys_per_p, y_ikeys)

For p=0, 69% - 15793/22921 was recovered in top 1
For p=0.01, 67% - 15459/22921 was recovered in top 1
For p=0.03, 65% - 14823/22921 was recovered in top 1
For p=0.05, 62% - 14109/22921 was recovered in top 1
For p=0.08, 58% - 13209/22921 was recovered in top 1
For p=0.1, 55% - 12624/22921 was recovered in top 1
For p=0.15, 48% - 11115/22921 was recovered in top 1
For p=0.2, 42% - 9597/22921 was recovered in top 1
For p=0.3, 30% - 6778/22921 was recovered in top 1
For p=0.5, 12% - 2797/22921 was recovered in top 1


### Weighted cosine

In [58]:
tested = "spec_val"
p_name = "cosine_weighted"

n_pred_ikeys_per_p = np.load(f"matches/{kind}/{dataset}/{tested}/{p_name}/n_pred_ikeys_per_p.npy")
n_pred_ikeys_per_k = np.load(f"matches/{kind}/{dataset}/{tested}/{p_name}/n_pred_ikeys_per_k.npy", allow_pickle=True)
y_ikeys = np.load(f"matches/{kind}/{dataset}/{tested}/{p_name}/y_ikeys.npy")

In [59]:
show_for_top(1, n_pred_ikeys_per_k, y_ikeys)

For k=1, 0% - 50/22921 was recovered in top 1
For k=2, 5% - 1039/22921 was recovered in top 1
For k=3, 13% - 3025/22921 was recovered in top 1
For k=4, 22% - 5134/22921 was recovered in top 1
For k=5, 30% - 6962/22921 was recovered in top 1
For k=6, 37% - 8574/22921 was recovered in top 1
For k=7, 43% - 9841/22921 was recovered in top 1
For k=8, 48% - 10928/22921 was recovered in top 1
For k=9, 52% - 11872/22921 was recovered in top 1
For k=10, 55% - 12641/22884 was recovered in top 1
For k=11, 58% - 13292/22854 was recovered in top 1
For k=12, 61% - 13870/22818 was recovered in top 1
For k=13, 63% - 14297/22771 was recovered in top 1
For k=14, 65% - 14711/22714 was recovered in top 1
For k=15, 67% - 15062/22640 was recovered in top 1
For k=16, 68% - 15339/22575 was recovered in top 1
For k=17, 69% - 15598/22507 was recovered in top 1
For k=18, 70% - 15758/22415 was recovered in top 1
For k=19, 71% - 15910/22305 was recovered in top 1
For k=20, 72% - 16058/22206 was recovered in top 1


In [60]:
show_for_random(1, probs, n_pred_ikeys_per_p, y_ikeys)

For p=0, 84% - 19202/22921 was recovered in top 1
For p=0.01, 83% - 18959/22921 was recovered in top 1
For p=0.03, 80% - 18402/22921 was recovered in top 1
For p=0.05, 78% - 17932/22921 was recovered in top 1
For p=0.08, 75% - 17154/22921 was recovered in top 1
For p=0.1, 73% - 16655/22921 was recovered in top 1
For p=0.15, 67% - 15287/22921 was recovered in top 1
For p=0.2, 60% - 13838/22921 was recovered in top 1
For p=0.3, 48% - 10889/22921 was recovered in top 1
For p=0.5, 23% - 5197/22921 was recovered in top 1


## Evaluate the test set

In [17]:
# parameters
up_to_k = 30

probs = [0, .05, .1, .15, .2, .25, .3, .35, .4, .45, .5]

to_match = 50


In [18]:
matcher = BaseMatcherKNN(sparse_mz_matrix, max_mz=max_mz, is_weighted=False)

tested = "spec_test"
p_name = "cosine"

In [19]:
%%time
n_pred_ikeys_per_k, y_ikeys = matcher.match_topk_all(datasets["spec_train"], datasets[tested], \
                                                     up_to_k, to_match)
np.save(f"matches/{kind}/{dataset}/{tested}/{p_name}/n_pred_ikeys_per_k.npy", n_pred_ikeys_per_k)
np.save(f"matches/{kind}/{dataset}/{tested}/{p_name}/y_ikeys.npy", y_ikeys)

/opt/conda/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


CPU times: user 8h 10min 11s, sys: 59min 3s, total: 9h 9min 14s
Wall time: 9h 9min 20s


In [20]:
%%time
n_pred_ikeys_per_p, y_ikeys = matcher.match_random_all(datasets["spec_train"], datasets[tested], \
                                                       probs, to_match)
np.save(f"matches/{kind}/{dataset}/{tested}/{p_name}/n_pred_ikeys_per_p.npy", n_pred_ikeys_per_p)

CPU times: user 8h 58min 22s, sys: 47min 26s, total: 9h 45min 48s
Wall time: 3h 24min 32s


In [21]:
matcher_weighted = BaseMatcherKNN(sparse_weighted_mz_matrix, max_mz=max_mz, is_weighted=True)

tested = "spec_test"
p_name = "cosine_weighted"

In [22]:
%%time
n_pred_ikeys_per_k, y_ikeys = matcher_weighted.match_topk_all(datasets["spec_train"], \
                                                              datasets[tested], up_to_k, to_match)
np.save(f"matches/{kind}/{dataset}/{tested}/{p_name}/n_pred_ikeys_per_k.npy", n_pred_ikeys_per_k)
np.save(f"matches/{kind}/{dataset}/{tested}/{p_name}/y_ikeys.npy", y_ikeys)


CPU times: user 8h 22min 50s, sys: 58min 36s, total: 9h 21min 27s
Wall time: 9h 21min 33s


In [23]:
%%time
n_pred_ikeys_per_p, y_ikeys = matcher_weighted.match_random_all(datasets["spec_train"], \
                                                                datasets[tested], probs, to_match)
np.save(f"matches/{kind}/{dataset}/{tested}/{p_name}/n_pred_ikeys_per_p.npy", n_pred_ikeys_per_p)

CPU times: user 8h 59min 58s, sys: 48min 33s, total: 9h 48min 32s
Wall time: 3h 27min 14s
